* ライブラリ比較関数というスペクトルデータを解析する関数になります。
    - 鉱物スペクトルデータを収集したライブラリファイルと調査したい対象スペクトルデータ１つを比較します。
    - ライブラリファイルには、約6000のスペクトルデータがあります。
    - それぞれ比較し、類似度を算出することで、類似する鉱物を同定します。
* 「いじってみて」と記述されている直下のセルをいじってみてください。
* 実行は、Shift＋Enter です。
* いじってみた後に以下の項目それぞれの感想（評価）をください。なんでも大丈夫です。お願いします。
    - 良いところ
    - 悪いところ（良くないところ）
    - もう少しなところ（あれば良いと思うところ）
    - 他のアイデア（あればでいいです）

## ライブラリ比較関数

In [11]:
import pandas as pd
import importlib
import libcomp as lc
importlib.reload(lc)

# ライブラリファイル指定
library_file = 'RelabDatabase2022Dec31_mini/Library_Mineral_BD-VNIR.csv'
# カタログファイル指定
sample_catalogue_file = 'RelabDatabase2022Dec31_mini/catalogues/Sample_Catalogue.xlsx'
# 対象スペクトルファイル指定
target_spectrum_file = './data/frt00003621_07_if166l_E_-97.57339 N_24.96872.csv'

* いじってみて↓

In [ ]:
''' 類似度算出に使用する各タイプ指定
        以下のいずれかのタイプを選択する。
            interp_type :   liner, sp
            scaling_type:   norm, st
            pcc         :   pcc, cos, edis     
        各タイプの説明
            interp_type     >> liner(線形補間), sp(スプライン補間)
            scaling_type    >> norm(正規化), st(標準化)
            pcc             >> pcc(ピアソンの相関係数), cos(コサイン類似度), edis(ユークリッド距離)
'''
interp_type, scaling_type, sim_type = 'liner', 'st', 'pcc'

In [ ]:
comparer = lc.LibraryComparison(library_file, sample_catalogue_file, target_spectrum_file)  # インスタンス作成

### 類似度判定ステップ
* いじってみて↓

In [ ]:
''' 対象とする波長範囲（index）を指定
        comparer.slice_wav(100, 200)    >> 波長の100〜200を使用する
        comparer.slice_wav(100)         >> 波長の100〜を使用する
'''
# comparer.slice_wav(100, 200)  # 対象とする波長範囲（index）

In [ ]:
comparer.interpolate(interp_type)
comparer.scaling(scaling_type)
comparer.measure_similarity(sim_type)
comparer.result()

# result.to_excel('libcomp_result.xlsx', index=False)  # excelファイル作成

### プロットステップ

In [9]:
comparer.find_common_valley_indices()  # 共通の谷点のインデックスを取得
df = comparer.result()
target_df = df.query("index == 0")
library_df = df.query("index != 0")
library_df2 = library_df.copy()

100%|██████████| 6359/6359 [00:00<00:00, 94955.85 count/s]

>>> Filtering(common valleys) completed.



* いじってみて↓

In [10]:
''' フィルター
        以下、フィルターをかけたいコードを実行する。（好きなように）
'''
library_df2 = library_df2[library_df2['Band'] > 50]  # 一致バンド数が５０以上

''' ソート
        以下のいずれかを実行する。（実行行だけコメント解除する）
            類似度、一致バンド数
        引数説明
            by          >> データフレームの列名
            ascending   >> True(降順), False(昇順)
'''
library_df2 = library_df2.sort_values(by=sim_type, ascending=False)  # 類似度
# library_df2 = library_df2.sort_values(by="Band", ascending=False)  # 一致バンド数

In [ ]:
library_df2 = pd.concat([target_df, library_df2])  # 対象スペクトルとライブラリスペクトルのデータフレーム結合

#### プロット
* いじってみて↓

In [ ]:
''' 指定したインデックスをプロット
        index_list: 指定したインデックスのスペクトルがプロットされる
'''
index_list = [0]

''' 指定されたインデックスをプロット
        引数説明
            avg_size    : 移動平均を取る値
            find        : valley(谷点を表示), None(谷点を表示しない)
            prominence  : 谷点の判断基準の値

'''
comparer.plot(library_df2, index_list, 'raw', avg_size=3, find='valley', prominence=0.01)

''' 指定した上位の数分プロット
        以下のいずれかを実行する。（実行行だけコメント解除する）
            nlargest, nsmallest
        各関数の説明
            nlargest    >> 大きい方から
            nsmallest   >> 小さい方から
'''
index_list = library_df2.nlargest(10, sim_type).index
# index_list = library_df2.nsmallest(10, sim_type).index

comparer.plot(library_df2, index_list, 'raw', find='valley', prominence=0.01)